In [587]:
import pandas as pd
import numpy as np

In [687]:
hosp = pd.read_csv('Hospitals_geocoded.csv')

#### Section Used to Manually Obtain Latitude and Longitude for Hospitals
* no longer need to be run
---------------------------------------------------------------------------

In [641]:
hosp_missing = hosp[hosp.latitude.isna()][['Facility_Name','Address','City']].reset_index().copy()
hosp_missing['latitude'] = [np.NaN for n in range(hosp_missing.shape[0])]
hosp_missing['longitude'] = [np.NaN for n in range(hosp_missing.shape[0])]

hosp_missing
missing = hosp_missing['Facility_Name']+', '+hosp_missing.Address+', '+hosp_missing.City+', USA'

In [642]:
import geopy
from geopy import Nominatim

locator = Nominatim(user_agent="nycDSA")

In [643]:
current = 0
missing[current]

"Children'S National Hospital, 111 MICHIGAN AVE, NW, WASHINGTON, USA"

In [648]:
lat = locator.geocode("2010 HEALTH CAMPUS DRIVE, HARRISONBURG, USA")[1][0]
long = locator.geocode("2010 HEALTH CAMPUS DRIVE, HARRISONBURG, USA")[1][1]
print(f'{lat}, {long}')

38.4051275, -78.85520746693862


In [645]:
hosp_missing.loc[current,'latitude'] = lat
hosp_missing.loc[current,'longitude'] = long

In [646]:
hosp_missing.loc[[current]]

index                 Facility_Name               Address        City  \
0      6  Children'S National Hospital  111 MICHIGAN AVE, NW  WASHINGTON   

    latitude  longitude  
0  38.927021 -77.013979


---------------------------------------------------------------------------

In [686]:
#hosp.to_csv('Hospitals_geocoded.csv', index=False)

In [680]:
correction = "Thomas B Finan Center"
hosp[hosp.Facility_Name==correction]

Facility_Name                                         Address  \
416  Thomas B Finan Center  10102 COUNTRY CLUB ROAD,  POST OFFICE BOX 1722   

           City State  ZIP_Code County_Name    Phone_Number Hospital_Type  \
416  CUMBERLAND    MD     21502    Allegany  (301) 777-2240   Psychiatric   

     Hospital_Ownership Emergency_Services Overall_Rating   latitude  \
416  Government - Local                 No  Not Available  40.087879   

     longitude  
416 -74.627962

In [683]:
lat = locator.geocode("12500 Willowbrook Rd, CUMBERLAND, USA")[1][0]
long = locator.geocode("12500 Willowbrook Rd, CUMBERLAND, USA")[1][1]
print(f'{lat}, {long}')

39.6572208, -78.748512


In [684]:
hosp.loc[(hosp.Facility_Name==correction),'latitude'] = lat
hosp.loc[(hosp.Facility_Name==correction),'longitude'] = long

In [688]:
# Check LAT_LONG on Map

import plotly.express as px
import plotly.graph_objects as go
# import plotly.offline as pyo
# pyo.init_notebook_mode()

fig = px.scatter_geo(hosp, lat="latitude", 
                     lon="longitude",
                    hover_name="Facility_Name")

#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 🏥 Quick EDA

In [592]:
hosp.columns

Index(['Facility_Name', 'Address', 'City', 'State', 'ZIP_Code', 'County_Name',
       'Phone_Number', 'Hospital_Type', 'Hospital_Ownership',
       'Emergency_Services', 'Overall_Rating', 'latitude', 'longitude'],
      dtype='object')

In [593]:
print('-----------------------------------')
print(hosp.Hospital_Type.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Emergency_Services.value_counts())
print('\n')

-----------------------------------
Acute Care Hospitals                  553
Critical Access Hospitals              92
Psychiatric                            83
Acute Care - Department of Defense     12
Childrens                              11
Name: Hospital_Type, dtype: int64


-----------------------------------
Yes    640
No     111
Name: Emergency_Services, dtype: int64




#### Remove <span style="color:darkred">Psychiatric</span> & <span style="color:red">non-ER</span> Hospitals

In [594]:
hosp = hosp[(hosp.Hospital_Type!='Psychiatric') & (hosp.Emergency_Services=='Yes')]

In [595]:
print('-----------------------------------')
print(f'Total ZIP codes: {hosp.ZIP_Code.nunique()}')
print('\n')
print('-----------------------------------')
print('# Hospitals by State')
print(hosp.State.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Overall_Rating.value_counts())

-----------------------------------
Total ZIP codes: 611


-----------------------------------
# Hospitals by State
FL    170
GA    126
NC    103
VA     80
SC     55
WV     46
MD     43
DC      6
Name: State, dtype: int64


-----------------------------------
3                175
4                139
2                136
Not Available     94
5                 52
1                 33
Name: Overall_Rating, dtype: int64


## Stanley Martin Homes Target ZIP codes and STATES

In [596]:
smh = pd.read_csv('redfin_scrapy/smh_zipcodes.csv', index_col='ZipCode')
smh.State.value_counts()

FL    1464
VA    1213
NC    1079
GA     950
WV     851
MD     603
SC     534
DC     293
Name: State, dtype: int64

In [597]:
import json
with open('C:/Users/dn-83/Documents/bootcamp/capstone_proj/_data/dc_district_of_columbia_zip_codes_geo.min.json') as f:
    dc_geodata = json.load(f)

In [598]:
dc_geodata

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'STATEFP10': '11',
    'ZCTA5CE10': '20024',
    'GEOID10': '1120024',
    'CLASSFP10': 'B5',
    'MTFCC10': 'G6350',
    'FUNCSTAT10': 'S',
    'ALAND10': 6770280,
    'AWATER10': 2003315,
    'INTPTLAT10': '+38.8760098',
    'INTPTLON10': '-077.0253225',
    'PARTFLG10': 'N'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-77.064251, 38.888956],
       [-77.064311, 38.889104],
       [-77.064325, 38.889146],
       [-77.064354, 38.889255],
       [-77.064365, 38.889287],
       [-77.064377, 38.889325],
       [-77.064384, 38.88936],
       [-77.064393, 38.889408],
       [-77.064402, 38.889442],
       [-77.064409, 38.889473],
       [-77.064414, 38.889502],
       [-77.064418, 38.889535],
       [-77.064415, 38.889571],
       [-77.064409, 38.889598],
       [-77.064389, 38.889643],
       [-77.064378, 38.889659],
       [-77.064363, 38.889677],
       [-77.064339, 38.889712],
       [

In [624]:
import plotly.express as px
import plotly.graph_objects as go
# import plotly.offline as pyo
# pyo.init_notebook_mode()

# fig = go.Figure(
#     go.Choroplethmapbox(
#         meta = smh[smh.State=='DC'], 
#         locations=smh[smh.State=='DC'].index,
#         geojson=dc_geodata, 
#         featureidkey='properties.ZCTA5CE10'
#     ))

fig = px.choropleth(data_frame = smh[smh.State=='DC'], 
                    locations=smh[smh.State=='DC'].index,
                    geojson=dc_geodata, 
                    fitbounds='locations',
                    featureidkey='properties.ZCTA5CE10')

fig.add_scattergeo(lat=hosp[hosp.State=='DC'].latitude, 
                      lon=hosp[hosp.State=='DC'].longitude,
                   text=hosp[hosp.State=='DC'].Facility_Name,
                      marker_size=2)

#fig = px.scatter_geo(hosp[hosp.State=='DC'], lat="latitude", lon="longitude")

#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [581]:
hosp[hosp.State=='DC']

Facility_Name                   Address  \
0         George Washington Univ Hospital            900 23RD ST NW   
2  Medstar Georgetown University Hospital         3800 RESERVOIR RD   
3                Sibley Memorial Hospital      5255 LOUGHBORO RD NW   
4                   United Medical Center  1310 SOUTHERN AVENUE  SE   
5      Medstar Washington Hospital Center      110 IRVING STREET NW   
6            Children'S National Hospital      111 MICHIGAN AVE, NW   

         City State  ZIP_Code           County_Name    Phone_Number  \
0  WASHINGTON    DC     20037  District of Columbia  (202) 716-4605   
2  WASHINGTON    DC     20007  District of Columbia  (202) 784-3000   
3  WASHINGTON    DC     20016  District of Columbia  (202) 537-4680   
4  WASHINGTON    DC     20032  District of Columbia  (202) 574-6611   
5  WASHINGTON    DC     20010  District of Columbia  (202) 877-7000   
6  WASHINGTON    DC     20010  District of Columbia  (202) 476-5000   

          Hospital_Type              Hospital_Ownership Emergency_Services  \
0  Acute Care Hospitals                     Proprietary                Yes   
2  Acute Care Hospitals                     Proprietary                Yes   
3  Acute Care Hospitals    Voluntary non-profit - Other                Yes   
4  Acute Care Hospitals                     Proprietary                Yes   
5  Acute Care Hospitals    Voluntary non-profit - Other                Yes   
6             Childrens  Voluntary non-profit - Private                Yes   

  Overall_Rating   latitude   longitude  
0              1  38.901411  -77.050600  
2              2  46.074927 -118.279273  
3              4  38.936770  -77.108880  
4              1  38.835876  -76.984938  
5              3  38.928749  -77.014769  
6  Not Available        NaN         NaN